In [73]:
import pandas as pd
import math
import ast
import numpy as np
import re

Store the data in two lists that contains the different lines

In [74]:
with open('C:/Users/juan_/Desktop/programación/NLP_ML_DL_UdemyCourse/documents_for_course/Edgar_allan_poe.txt',encoding="utf-8", mode = "r") as doc:
    poe_dataset_noisy = doc.readlines()

In [75]:
with open('C:/Users/juan_/Desktop/programación/NLP_ML_DL_UdemyCourse/documents_for_course/Robert_frost.txt',encoding="utf-8", mode = "r") as doc:
    frost_dataset_noisy = doc.readlines()

We're gonna preprocess the text so it doesn't take 'The' or 'the' as different states cause they are actually the same. Also we will take a look at punctuation and remove it as well as the empty lines

In [76]:
def clean_dataset(dataset):
    clean_dataset = []
    for line in dataset:
        line = line.lower()
        line = re.sub(r"([-.,\\/#!$%\\^&\\*;:{}=\\-_`~()])","",line)
        line = re.sub(r"\n","",line)
        line = line.strip()
        clean_dataset.append(line)
    return clean_dataset

In [77]:
poe_dataset = clean_dataset(poe_dataset_noisy)
frost_dataset = clean_dataset(frost_dataset_noisy)

Create a function that generates a dataframe for each author and split that dataframe into testing and training 

In [78]:
def divide_set(dataset):
    dataframe_set = pd.DataFrame({'Author': pd.Series(dtype='str'),'Poem': pd.Series(dtype='str')})
    for sentence in range(len(dataset)):
        list_row = ["Poe",dataset[sentence]]
        dataframe_set.loc[len(dataframe_set)] = list_row
        

    training_data = dataframe_set.sample(frac=0.8, random_state=25)
    testing_data = dataframe_set.drop(training_data.index)

    return training_data, testing_data


Generate with our function out training and testing datasets

In [79]:
poe_training_data, poe_testing_data = divide_set(poe_dataset)
frost_training_data, frost_testing_data = divide_set(frost_dataset)
# print(poe_training_data.head())

Create a function that, given a specific dataframe, returns a word to index mapping for every word in the dataset

In [80]:
def get_vocabulary_size(dataset):
    number_of_states = {}
    counter = 0
    for index, row in dataset.iterrows():
        poem_line = row['Poem']
        for word in poem_line.split(" "):
            if word not in number_of_states:
                number_of_states[word] = counter
                counter +=1
    number_of_states["unknown_word"] = len(number_of_states) + 1
    return number_of_states

In [81]:
states_in_poe = get_vocabulary_size(poe_training_data)
states_in_frost = get_vocabulary_size(frost_training_data)
# print(states_in_poe)

Now we have to turn every line of text in the training dataset into its respective series of integers from our mapping. 
Then we create a markov model for each of the classes

In [59]:
def transform_dataset_to_index_mapping(dataset, states_map):
    dataframe_indexed_set = pd.DataFrame({'Line': pd.Series(dtype=int),'Map': pd.Series(dtype=object)})
    counter = 0
    for index, row in dataset.iterrows():
        line_of_strings = row['Poem']
        line_of_indexes = []
        for word in line_of_strings.split(" "):
            if word in states_map.keys():
                line_of_indexes.append(states_map[word])
        dataframe_indexed_set.loc[counter] = [index,line_of_indexes]
        counter +=1
    return dataframe_indexed_set
        


In [60]:
poe_traning_indexed_df = transform_dataset_to_index_mapping(poe_training_data, states_in_poe)
poe_test_indexed_df = transform_dataset_to_index_mapping(poe_testing_data, states_in_poe)

frost_traning_indexed_df = transform_dataset_to_index_mapping(frost_training_data, states_in_frost)
frost_traning_indexed_df = transform_dataset_to_index_mapping(frost_training_data, states_in_frost)


In [61]:
poe_traning_indexed_df.head()

,Line,Map
0,270,"[0, 1, 2, 3, 4, 5, 6, 7]"
1,74,"[8, 9, 10, 11, 12]"
2,103,"[13, 14, 15, 1, 16, 17, 18, 19, 20, 21]"
3,262,[22]
4,469,"[23, 24, 25]"


Now we need to, for each dataset, train our markov model. In order to do that we have to create out matrix A and our vector pi. so
1st: we create a function that will generate a df correspondent to our matrix A with empty value. By now we only want the mould.
2nd: we create the transition matrix using the theory that we've learnt in the course. This will take some time
3rd: we also create a function for the initial state probability
4th: Now we need to group all these 3 processes and we will have a function that creates our markov model for a given dataset. 

With these done we're able to make predictions in a given sequence

In [62]:
def create_skeleton_transitions_matrix(states):
    number_of_states = list(states.values())
    number_of_states.append('unknown')
    dictionary_dataframe = {}
    for state in number_of_states:
        dictionary_dataframe[state] = 1
    dictionary_dataframe['unknown'] = 1
    dataframe_transiciones_estados = pd.DataFrame(dictionary_dataframe, index=number_of_states)
    dataframe_initial_vector = pd.DataFrame(number_of_states)

    return dataframe_transiciones_estados, dataframe_initial_vector, len(number_of_states) 

In [69]:
skeleton_A_poe = create_skeleton_transitions_matrix(states_in_poe)
skeleton_A_frost = create_skeleton_transitions_matrix(states_in_frost)

In [70]:
def create_transition_matrix_and_initial_vector(sequences, matrix_skeleton, pi_vector):
    # Dataset debe ser la columna map del df que contiene los índices
    # Poe_training_indexed_df es el df que contiene las secuencias por indices [frase [1,2,3]]
    # Possible states es el diccionario 'palabra':indice ('but':0) poe_states.value    
    # dataframe_estados = create_skeleton_transitions_matrix(dataset)
    
    for sequence in sequences.iloc[:,1]:
        for index, element in enumerate(sequence):
            if index == 0:
                pi_vector.at[element,0] +=1
            else:
                previous_state = sequence[index-1]
                matrix_skeleton.at[previous_state,element] +=1
    # Now we have the vector as well as the initial state for the model with the add one smoothing but we need to make the division at each row
    transition_matrix = pd.DataFrame()
    for index, row in matrix_skeleton.iterrows():
        # In this case we don't need to add to the denominator anythong because the number of states is already there thanks to the add one smoothing
        denominator = np.sum(row,axis=0)
        new_row = row/denominator
        transition_matrix[index] = new_row

    return transition_matrix, pi_vector

In [71]:
markov_poe = create_transition_matrix_and_initial_vector(poe_traning_indexed_df,skeleton_A_poe[0],skeleton_A_poe[1])


C:\Users\juan_\AppData\Local\Temp\ipykernel_1324\1777347270.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  transition_matrix[index] = new_row
C:\Users\juan_\AppData\Local\Temp\ipykernel_1324\1777347270.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  transition_matrix[index] = new_row
C:\Users\juan_\AppData\Local\Temp\ipykernel_1324\1777347270.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all col

In [72]:
markov_frost = create_transition_matrix_and_initial_vector(frost_traning_indexed_df, skeleton_A_frost[0],skeleton_A_frost[1])


C:\Users\juan_\AppData\Local\Temp\ipykernel_1324\1777347270.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  transition_matrix[index] = new_row
C:\Users\juan_\AppData\Local\Temp\ipykernel_1324\1777347270.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  transition_matrix[index] = new_row
C:\Users\juan_\AppData\Local\Temp\ipykernel_1324\1777347270.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all col

In [91]:
def get_sentence_probability(sentence, model, index_mapping):
    splitted_sentence = sentence.split(" ")
    trasnformed_to_index_mapping = []
    for word in splitted_sentence:
        if word in index_mapping:
            trasnformed_to_index_mapping.append(index_mapping[word])
        else:
            trasnformed_to_index_mapping.append('unknown')
    
    for state in trasnformed_to_index_mapping:
        # Coger el valor del vector inicial para la primera palabra y los indices del df para las siguientes y después ejecutar la probabilidad
        pass

In [92]:
get_sentence_probability('Of the old time entombed','hola', states_in_poe)

['unknown', 16, 128, 129, 808]
hola


In [ ]:
# def classify_sentence(sentence):
#     probability_poe = get_sentence_probability(poe_dataset,sentence)
#     probability_frost = get_sentence_probability(frost_dataset,sentence)
#     probability_poe = math.log(probability_poe)
#     probability_frost = math.log(probability_frost)
#     print(probability_poe)
#     print(probability_frost)
#     if probability_poe > probability_frost:
#         print("El texto es de poe")
#     else:
#         print("El texto es de frost")

In [ ]:
# classify_sentence("""And travellers now within that valley,
# Through the red-litten windows, see
# Vast forms that move fantastically
# To a discordant melody;
# While, like a rapid ghastly river,
# Through the pale door,
# A hideous throng rush out forever,
# And laugh --but smile no more.""")